<a href="https://colab.research.google.com/github/heejjj/Health_Bio_AI/blob/main/0518_fine_tuning_CBHipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers==4.10.2
!pip install --upgrade accelerate
!pip install -v --no-cache-dir --force-reinstall tokenizers -f https://huggingface.co/distilgpt2/tree/main/tokenizers/dist/
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 46.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 25.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=6db3414c6cd3a8149af00bedfeb001e11d03ea7bcb31829ebdb3109b9ed8d176
  Stored in directory: /r

In [ ]:
!pip install kogpt2-transformers
!pip install --upgrade kogpt2-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 챗봇 자체 데이터 파인튜닝 

# 아래 코드는 한국어 GPT2 모델을 파인 튜닝하여 대화형 챗봇을 만드는 코드입니다.


# 코드의 구성은 다음과 같습니다.


# 라이브러리 임포트 - pytorch, transformers, kogpt2_transformers
# 학습 데이터 정의
# Kogpt 모델 및 tokenizer 로드
# 특수 토큰 추가
# 파인튜닝을 위한 데이터셋 클래스 정의
# 하이퍼파라미터 설정 - max_length, batch_size, epochs, learning_rate
# 데이터셋 및 데이터로더 생성
# GPU 사용 여부 확인
# 옵티마이저 설정
# 파인튜닝 시작
# 파인튜닝된 모델 저장



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2Config, AdamW
from kogpt2_transformers import get_kogpt2_model, get_kogpt2_tokenizer

# 학습 데이터
train_data = [
    ("안녕하세요?", "안녕하세요!"),
    ("배고파요", "뭘 먹을까요?"),
    ("오늘 날씨가 어때요?", "오늘은 맑은 날씨입니다.")
    
]

# Kogpt 모델 및 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = get_kogpt2_tokenizer()
model = get_kogpt2_model(model_name)

# 특수 토큰 추가
tokenizer.add_tokens(['<USER>', '<SYSTEM>'])
model.resize_token_embeddings(len(tokenizer))

# 파인튜닝을 위한 데이터셋 클래스 정의
class ChatDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text, target_text = self.data[idx]

        encoded_input = self.tokenizer.encode(input_text, add_special_tokens=True)
        encoded_target = self.tokenizer.encode(target_text, add_special_tokens=True)

        # 길이가 초과하는 경우 자르고 패딩 추가
        if len(encoded_input) > self.max_length:
            encoded_input = encoded_input[:self.max_length]
        else:
            encoded_input += [tokenizer.pad_token_id] * (self.max_length - len(encoded_input))

        if len(encoded_target) > self.max_length:
            encoded_target = encoded_target[:self.max_length]
        else:
            encoded_target += [tokenizer.pad_token_id] * (self.max_length - len(encoded_target))

        return torch.tensor(encoded_input), torch.tensor(encoded_target)

# 하이퍼파라미터 설정
max_length = 128
batch_size = 1
epochs = 30
learning_rate = 1e-4

# 데이터셋 및 데이터로더 생성
dataset = ChatDataset(train_data, tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=learning_rate)

# 파인튜닝 시작
model.train()
for epoch in range(epochs):
    total_loss = 0.0

    for inputs, targets in dataloader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs, labels=targets)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Avg. Loss: {avg_loss:.4f}")

# 파인튜닝된 모델 저장
save_path = "kogpt_chatbot_finetuned.pth"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/30 - Avg. Loss: 7.1204
Epoch 2/30 - Avg. Loss: 0.3170
Epoch 3/30 - Avg. Loss: 0.2897
Epoch 4/30 - Avg. Loss: 0.2294
Epoch 5/30 - Avg. Loss: 0.1721
Epoch 6/30 - Avg. Loss: 0.1096
Epoch 7/30 - Avg. Loss: 0.1000
Epoch 8/30 - Avg. Loss: 0.0789
Epoch 9/30 - Avg. Loss: 0.0489
Epoch 10/30 - Avg. Loss: 0.0366
Epoch 11/30 - Avg. Loss: 0.0283
Epoch 12/30 - Avg. Loss: 0.0258
Epoch 13/30 - Avg. Loss: 0.0275
Epoch 14/30 - Avg. Loss: 0.0179
Epoch 15/30 - Avg. Loss: 0.0136
Epoch 16/30 - Avg. Loss: 0.0090
Epoch 17/30 - Avg. Loss: 0.0072
Epoch 18/30 - Avg. Loss: 0.0072
Epoch 19/30 - Avg. Loss: 0.0057
Epoch 20/30 - Avg. Loss: 0.0085
Epoch 21/30 - Avg. Loss: 0.0047
Epoch 22/30 - Avg. Loss: 0.0063
Epoch 23/30 - Avg. Loss: 0.0030
Epoch 24/30 - Avg. Loss: 0.0049
Epoch 25/30 - Avg. Loss: 0.0031
Epoch 26/30 - Avg. Loss: 0.0027
Epoch 27/30 - Avg. Loss: 0.0022
Epoch 28/30 - Avg. Loss: 0.0084
Epoch 29/30 - Avg. Loss: 0.0013
Epoch 30/30 - Avg. Loss: 0.0018


('kogpt_chatbot_finetuned.pth/tokenizer_config.json',
 'kogpt_chatbot_finetuned.pth/special_tokens_map.json',
 'kogpt_chatbot_finetuned.pth/tokenizer.json')

In [ ]:
import torch
from transformers import GPT2LMHeadModel
from kogpt2_transformers import get_kogpt2_tokenizer

# 저장된 모델 로드
save_path = "kogpt_chatbot_finetuned.pth"
model = GPT2LMHeadModel.from_pretrained(save_path)

# 토크나이저 로드
tokenizer = get_kogpt2_tokenizer()

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 대화하기
while True:
    # 사용자로부터 질문 입력
    question = input("질문을 입력하세요 (종료하려면 '종료' 입력): ")

    if question == "종료":
        break

    # 입력 문장 토큰화
    input_ids = tokenizer.encode(question, return_tensors="pt").to(device)

    # 모델에 입력 전달하여 답변 생성
    output = model.generate(
        input_ids,
        max_length=128,
        num_return_sequences=1,
        temperature=0.5,  # temperature 값을 0.5로 변경하여 더 다양한 답변을 생성합니다.
    )

    # 생성된 답변 디코딩 및 출력
    for i, answer in enumerate(output):
        answer = tokenizer.decode(answer, skip_special_tokens=True)
        print(f"답변 {i+1}: {answer}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


답변 1: 전립선 암이란 무엇인가요?
답변 1: 오늘 날씨가 어때요?
답변 1: 배고파요?


In [ ]:
from datasets import load_dataset

#dataset = load_dataset('데이터셋 이름')
dataset = load_dataset('daekeun-ml/naver-news-summarization-ko')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/daekeun-ml___csv/daekeun-ml--naver-news-summarization-ko-884ccea06154613b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]